In [28]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
#stemmer
stemmer = LancasterStemmer()

In [29]:
# 2 classes of training data
training_data = []
training_data.append({"class":"fund_transfer", "sentence":"transfer rs to "})
training_data.append({"class":"fund_transfer", "sentence":"do rs fund tranfer "})
training_data.append({"class":"fund_transfer", "sentence":"send rs amount to "})
training_data.append({"class":"fund_transfer", "sentence":"make rs payment to "})

training_data.append({"class":"goodbye", "sentence":"have a nice day"})
training_data.append({"class":"goodbye", "sentence":"see you later"})
training_data.append({"class":"goodbye", "sentence":"have a nice day"})
training_data.append({"class":"goodbye", "sentence":"talk to you soon"})

training_data.append({"class":"food", "sentence":"make me a sandwich"})
training_data.append({"class":"food", "sentence":"can you make a sandwich?"})
training_data.append({"class":"food", "sentence":"having a sandwich today?"})
training_data.append({"class":"food", "sentence":"what's for lunch?"})
print ("%s sentences of training data" % len(training_data))

12 sentences of training data


In [30]:
# capture unique stemmed words in the training corpus
corpus_words = {}
class_words = {}
# turn a list into a set (of unique items) and then a list again (this removes duplicates)
classes = list(set([a['class'] for a in training_data]))
for c in classes:
    # prepare a list of words within each class
    class_words[c] = []

# loop through each sentence in our training data
for data in training_data:
    # tokenize each sentence into words
    for word in nltk.word_tokenize(data['sentence']):
        # ignore a some things
        if word not in ["?", "'s"]:
            # stem and lowercase each word
            stemmed_word = stemmer.stem(word.lower())
            # have we not seen this word already?
            if stemmed_word not in corpus_words:
                corpus_words[stemmed_word] = 1
            else:
                corpus_words[stemmed_word] += 1

            # add the word to our words in class list
            class_words[data['class']].extend([stemmed_word])

# we now have each stemmed word and the number of occurances of the word in our training corpus (the word's commonality)
print ("Corpus words and counts: %s \n" % corpus_words)
# also we have all words in each class
print ("Class words: %s" % class_words)

Corpus words and counts: {'soon': 1, 'see': 1, 'today': 1, 'transf': 1, 'what': 1, 'for': 1, 'rs': 4, 'pay': 1, 'send': 1, 'to': 4, 'you': 3, 'hav': 3, 'do': 1, 'sandwich': 3, 'nic': 2, 'me': 1, 'fund': 1, 'lat': 1, 'mak': 3, 'day': 2, 'a': 5, 'lunch': 1, 'tranf': 1, 'amount': 1, 'can': 1, 'talk': 1} 

Class words: {'sandwich': ['mak', 'me', 'a', 'sandwich', 'can', 'you', 'mak', 'a', 'sandwich', 'hav', 'a', 'sandwich', 'today', 'what', 'for', 'lunch'], 'fund_transfer': ['transf', 'rs', 'to', 'do', 'rs', 'fund', 'tranf', 'send', 'rs', 'amount', 'to', 'mak', 'rs', 'pay', 'to'], 'goodbye': ['hav', 'a', 'nic', 'day', 'see', 'you', 'lat', 'hav', 'a', 'nic', 'day', 'talk', 'to', 'you', 'soon']}


In [31]:
# we can now calculate a score for a new sentence
sentence = "good day for us to have lunch?"

# calculate a score for a given class
def calculate_class_score(sentence, class_name, show_details=True):
    score = 0
    # tokenize each word in our new sentence
    for word in nltk.word_tokenize(sentence):
        # check to see if the stem of the word is in any of our classes
        if stemmer.stem(word.lower()) in class_words[class_name]:
            # treat each word with same weight
            score += 1
            
            if show_details:
                print ("   match: %s" % stemmer.stem(word.lower() ))
    return score

In [32]:
# now we can find the class with the highest score
for c in class_words.keys():
    print ("Class: %s  Score: %s \n" % (c, calculate_class_score(sentence, c)))

   match: for
   match: hav
   match: lunch
Class: sandwich  Score: 3 

   match: to
Class: fund_transfer  Score: 1 

   match: day
   match: to
   match: hav
Class: goodbye  Score: 3 



In [33]:
# calculate a score for a given class taking into account word commonality
def calculate_class_score_commonality(sentence, class_name, show_details=True):
    score = 0
    # tokenize each word in our new sentence
    for word in nltk.word_tokenize(sentence):
        # check to see if the stem of the word is in any of our classes
        if stemmer.stem(word.lower()) in class_words[class_name]:
            # treat each word with relative weight
            score += (1 / corpus_words[stemmer.stem(word.lower())])

            if show_details:
                print ("   match: %s (%s)" % (stemmer.stem(word.lower()), 1 / corpus_words[stemmer.stem(word.lower())]))
    return score

In [34]:
# now we can find the class with the highest score
for c in class_words.keys():
    print ("Class: %s  Score: %s \n" % (c, calculate_class_score_commonality(sentence, c)))

   match: for (1)
   match: hav (0)
   match: lunch (1)
Class: sandwich  Score: 2 

   match: to (0)
Class: fund_transfer  Score: 0 

   match: day (0)
   match: to (0)
   match: hav (0)
Class: goodbye  Score: 0 



In [35]:
# return the class with highest score for sentence
def classify(sentence):
    high_class = None
    high_score = 0
    # loop through our classes
    for c in class_words.keys():
        # calculate score of sentence for each class
        score = calculate_class_score_commonality(sentence, c, show_details=False)
        # keep track of highest score
        if score > high_score:
            high_class = c
            high_score = score

    return high_class, high_score

In [36]:
#test
classify("make me some lunch?")

('sandwich', 2)

In [37]:
classify("how are you doing today?")

('sandwich', 1)

In [38]:
classify("who are you?")

(None, 0)

In [39]:
classify("am I crazy?")

(None, 0)

In [40]:
classify("send 50 rs to bhavesh")

('fund_transfer', 1)